In [1]:
library(tidyverse)
library(haven)
library(dplyr)
library(scales)
library(tidyr)
library(sandwich)
library(stargazer)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    col_factor



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




In [2]:
my_data <- read.csv("censusdata.csv")

In [3]:
columns_data <- colnames(my_data)
columns_data

[1] "PPSORT"           "ABOID"            "AGEGRP"          
  [4] "AGEIMM"           "ATTSCH"           "BFNMEMB"         
  [7] "BedRm"            "CFInc"            "CFInc_AT"        
 [10] "CFSTAT"           "CHDBN"            "CIP2021"         
 [13] "CIP2021_STEM_SUM" "CMA"              "CONDO"           
 [16] "COVID_ERB"        "COW"              "CQPPB"           
 [19] "CapGn"            "CfSize"           "ChldC"           
 [22] "CitOth"           "Citizen"          "DIST"            
 [25] "DPGRSUM"          "DTYPE"            "EFDecile"        
 [28] "EFInc"            "EFInc_AT"         "EICBN"           
 [31] "ETHDER"           "EfDIMBM_2018"     "EfSize"          
 [34] "EmpIn"            "FOL"              "FPTWK"           
 [37] "Gender"           "GENSTAT"          "GovtI"           
 [40] "GTRfs"            "HCORENEED_IND"    "HDGREE"          
 [43] "HHInc"            "HHInc_AT"         "HHMRKINC"        
 [46] "HHSIZE"           "HHTYPE"           "HLMOSTEN"        
 [49] "HLMOSTFR"         "HLMOSTNO"         "HLREGEN"         
 [52] "HLREGFR"          "HLREGNO"          "IMMCAT5"         
 [55] "IMMSTAT"          "IncTax"           "Invst"           
 [58] "JOBPERM"          "KOL"              "LFACT"           
 [61] "LICO_BT"          "LICO_AT"          "LIPROGTYPE"      
 [64] "LI_ELIG_OML_U18"  "LOCSTUD"          "LOC_ST_RES"      
 [67] "LSTWRK"           "LWMOSTEN"         "LWMOSTFR"        
 [70] "LWMOSTNO"         "LWREGEN"          "LWREGFR"         
 [73] "LWREGNO"          "LoLIMA"           "LoLIMB"          
 [76] "LoMBM_2018"       "MODE"             "MTNEN"           
 [79] "MTNFR"            "MTNNO"            "MarStH"          
 [82] "Mob1"             "Mob5"             "MrkInc"          
 [85] "NAICS"            "NOC21"            "NOL"             
 [88] "NOS"              "OASGI"            "OtInc"           
 [91] "PKID25"           "PKID0_1"          "PKID15_24"       
 [94] "PKID2_5"          "PKID6_14"         "PKIDS"           
 [97] "POB"              "POBPAR1"          "POBPAR2"         
[100] "POWST"            "PR"               "PR1"             
[103] "PR5"              "PresMortG"        "PRIHM"           
[106] "PWDUR"            "PWLEAVE"          "PWOCC"           
[109] "PWPR"             "REGIND"           "Relig"           
[112] "REPAIR"           "ROOM"             "Retir"           
[115] "SHELCO"           "SSGRAD"           "Subsidy"         
[118] "SempI"            "Tenur"            "TotInc"          
[121] "TotInc_AT"        "VISMIN"           "Value"           
[124] "WKSWRK"           "WRKACT"           "Wages"           
[127] "YRIM"             "WEIGHT"           "WT1"             
[130] "WT2"              "WT3"              "WT4"             
[133] "WT5"              "WT6"              "WT7"             
[136] "WT8"              "WT9"              "WT10"            
[139] "WT11"             "WT12"             "WT13"            
[142] "WT14"             "WT15"             "WT16"

In [4]:
my_data <- my_data |> select(NAICS, PR, HDGREE, AGEGRP, NOC21, VISMIN) |> 
           filter(NAICS != 999 & NAICS != 888 & 
                  HDGREE != 88 & HDGREE != 99 & 
                  AGEGRP != 88, NOC21 != 88, 
                  NOC21 != 99, na.rm = TRUE)

head(my_data)

,NAICS,PR,HDGREE,AGEGRP,NOC21,VISMIN
,<int>,<int>,<int>,<int>,<int>,<int>
1,48,35,7,11,14,2
2,11,35,6,11,25,1
3,48,35,2,12,1,1
4,56,35,9,13,3,1
5,54,24,2,14,6,7
6,48,24,2,18,22,1


In [5]:
ai_data <- read.csv("AIusage.csv") |> rename(group = Business.characteristics)
head(ai_data)

,REF_DATE,GEO,DGUID,group,Use.of.artificial.intelligence..AI..by.businesses.or.organizations.in.producing.goods.or.delivering.services.over.the.last.12.months,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<lgl>,<int>
1,2025,Canada,2021A000011124,"North American Industry Classification System (NAICS), all industries","Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671082146,1.1.1,12.2,A,NA,NA,1
2,2025,Canada,2021A000011124,"Agriculture, forestry, fishing and hunting [11]","Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671082366,1.2.1,1.8,A,NA,NA,1
3,2025,Canada,2021A000011124,"Mining, quarrying, and oil and gas extraction [21]","Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671082586,1.3.1,5.6,A,NA,NA,1
4,2025,Canada,2021A000011124,Construction [23],"Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671082806,1.4.1,3.6,A,NA,NA,1
5,2025,Canada,2021A000011124,Manufacturing [31-33],"Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671083026,1.5.1,13.1,B,NA,NA,1
6,2025,Canada,2021A000011124,Wholesale trade [41],"Yes, business used AI for producing goods or delivering services over the last 12 months",Percent,239,units,0,v1671083246,1.6.1,10.6,B,NA,NA,1


In [6]:
ai_data <- ai_data %>% select(group, VALUE) |>
  filter(group %in% c('Agriculture, forestry, fishing and hunting [11]',
                   'Mining, quarrying, and oil and gas extraction [21]',
                   'Construction [23]',
                   'Manufacturing [31-33]',
                   'Wholesale trade [41]',
                   'Retail trade [44-45]',
                   'Transportation and warehousing [48-49]',
                   'Information and cultural industries [51]',
                   'Finance and insurance [52]',
                   'Real estate and rental and leasing [53]',
                   'Professional, scientific and technical services [54]',
                   'Administrative and support, waste management and remediation services [56]',
                   'Health care and social assistance [62]',
                   'Arts, entertainment and recreation [71]',
                   'Accommodation and food services [72]',
                   'Other services (except public administration) [81]'))

my_data <- my_data |> mutate(group = case_when(
    NAICS %in% c(11) ~ "Agriculture, forestry, fishing and hunting [11]",
    NAICS %in% c(21) ~ "Mining, quarrying, and oil and gas extraction [21]",
    NAICS %in% c(23) ~ "Construction [23]",
    NAICS %in% c(31, 32, 33) ~ "Manufacturing [31-33]",
    NAICS %in% c(41) ~ "Wholesale trade [41]",
    NAICS %in% c(44, 45) ~ "Retail trade [44-45]",
    NAICS %in% c(48, 49) ~ "Transportation and warehousing [48-49]",
    NAICS %in% c(51) ~ "Information and cultural industries [51]",
    NAICS %in% c(52) ~ "Finance and insurance [52]",
    NAICS %in% c(53) ~ "Real estate and rental and leasing [53]",
    NAICS %in% c(54) ~ "Professional, scientific and technical services [54]",
    NAICS %in% c(56) ~ "Administrative and support, waste management and remediation services [56]",
    NAICS %in% c(62) ~ "Health care and social assistance [62]",
    NAICS %in% c(71) ~ "Arts, entertainment and recreation [71]",
    NAICS %in% c(72) ~ "Accommodation and food services [72]",
    NAICS %in% c(81) ~ "Other services (except public administration) [81]",
    NAICS %in% c(22) ~ "Utilities [22]",
    NAICS %in% c(61) ~ "Education [61]",
    NAICS %in% c(91) ~ "Public Administration [91]"
  )) |>
  group_by(group) 

merged_data <- full_join(my_data,ai_data, by="group")
head(merged_data)

NAICS,PR,HDGREE,AGEGRP,NOC21,VISMIN,group,VALUE
<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>
48,35,7,11,14,2,Transportation and warehousing [48-49],1.8
11,35,6,11,25,1,"Agriculture, forestry, fishing and hunting [11]",1.8
48,35,2,12,1,1,Transportation and warehousing [48-49],1.8
56,35,9,13,3,1,"Administrative and support, waste management and remediation services [56]",9.8
54,24,2,14,6,7,"Professional, scientific and technical services [54]",31.7
48,24,2,18,22,1,Transportation and warehousing [48-49],1.8


In [16]:
regression1 = lm(VALUE ~ NOC21 + PR + HDGREE + AGEGRP, data = merged_data)

summary(regression1)

head(regression1$coefficients)


Call:
lm(formula = VALUE ~ NOC21 + PR + HDGREE + AGEGRP, data = merged_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-21.204  -6.178  -1.541   4.748  30.607 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 13.751472   0.336851  40.824  < 2e-16 ***
NOC21       -0.335601   0.007814 -42.951  < 2e-16 ***
PR          -0.018754   0.004429  -4.235  2.3e-05 ***
HDGREE       0.793782   0.017918  44.300  < 2e-16 ***
AGEGRP       0.004586   0.018781   0.244    0.807    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9.025 on 24916 degrees of freedom
  (4256 observations deleted due to missingness)
Multiple R-squared:  0.199,	Adjusted R-squared:  0.1989 
F-statistic:  1548 on 4 and 24916 DF,  p-value: < 2.2e-16


(Intercept)        NOC21           PR       HDGREE       AGEGRP 
13.751472279 -0.335600823 -0.018753506  0.793781525  0.004585842

In [17]:
regression2 = lm(VALUE ~ NOC21, 
                 data = merged_data)

summary(regression2)
 
head(regression2$coefficients)


Call:
lm(formula = VALUE ~ NOC21, data = merged_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.141  -6.375  -2.477   4.256  28.622 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 19.107984   0.118763  160.89   <2e-16 ***
NOC21       -0.466558   0.007464  -62.51   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9.375 on 24919 degrees of freedom
  (4256 observations deleted due to missingness)
Multiple R-squared:  0.1356,	Adjusted R-squared:  0.1355 
F-statistic:  3908 on 1 and 24919 DF,  p-value: < 2.2e-16


(Intercept)       NOC21 
 19.1079837  -0.4665582

In [18]:
regression3 = lm(VALUE ~ NOC21 + PR, 
                 data = merged_data)

summary(regression3)
 
head(regression3$coefficients)


Call:
lm(formula = VALUE ~ NOC21 + PR, data = merged_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-17.506  -6.339  -2.470   4.165  28.800 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 19.670754   0.207418  94.836  < 2e-16 ***
NOC21       -0.466930   0.007463 -62.566  < 2e-16 ***
PR          -0.015217   0.004599  -3.309 0.000937 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9.374 on 24918 degrees of freedom
  (4256 observations deleted due to missingness)
Multiple R-squared:  0.1359,	Adjusted R-squared:  0.1359 
F-statistic:  1960 on 2 and 24918 DF,  p-value: < 2.2e-16


(Intercept)       NOC21          PR 
19.67075393 -0.46692978 -0.01521747

In [19]:
regression4 = lm(VALUE ~ NOC21 + PR + HDGREE, 
                 data = merged_data)

summary(regression4)
 
head(regression4$coefficients)


Call:
lm(formula = VALUE ~ NOC21 + PR + HDGREE, data = merged_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-21.207  -6.189  -1.539   4.747  30.628 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 13.809297   0.239554  57.646  < 2e-16 ***
NOC21       -0.335799   0.007771 -43.211  < 2e-16 ***
PR          -0.018763   0.004428  -4.237 2.27e-05 ***
HDGREE       0.793814   0.017918  44.304  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 9.025 on 24917 degrees of freedom
  (4256 observations deleted due to missingness)
Multiple R-squared:  0.199,	Adjusted R-squared:  0.1989 
F-statistic:  2064 on 3 and 24917 DF,  p-value: < 2.2e-16


(Intercept)       NOC21          PR      HDGREE 
13.80929722 -0.33579896 -0.01876319  0.79381435

In [20]:
se <- sqrt(diag(vcovHC(regression1, type = "HC1")))

stargazer(regression2, regression3, regression4, regression1, 
          se = list(NULL, se), 
          type = "text", 
          column.labels = c("default", "robust"))


                                                                    Dependent variable:                                                
                    -------------------------------------------------------------------------------------------------------------------
                                                                           VALUE                                                       
                              default                       robust                                                                     
                                (1)                          (2)                          (3)                          (4)             
---------------------------------------------------------------------------------------------------------------------------------------
NOC21                        -0.467***                    -0.467***                    -0.336***                    -0.336***          
                              (0.007)          